In [22]:
import torch
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, GPT2Config, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

# File path
train_path = r"C:\Users\alvar\Desktop\UCM-TFM-G1\data\LLM\waste-classification\train.csv"
test_path = r"C:\Users\alvar\Desktop\UCM-TFM-G1\data\LLM\waste-classification\test.csv"
validation_path = r"C:\Users\alvar\Desktop\UCM-TFM-G1\data\LLM\waste-classification\validation.csv"

# Read the CSV files into DataFrames with the first column as the index
train_df = pd.read_csv(train_path, index_col=0)
test_df = pd.read_csv(test_path, index_col=0)
validation_df = pd.read_csv(validation_path, index_col=0)

In [23]:
# Tokenizer and Model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", padding=True, truncation=True, max_length=64)
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=len(label_encoder.classes_))

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Encoding labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df["Class"].tolist())
test_labels = label_encoder.transform(test_df["Class"].tolist())
val_labels = label_encoder.transform(validation_df["Class"].tolist())

# Tokenize the datasets
train_encodings = tokenizer(train_df["Phrase"].tolist(), padding=True, truncation=True)
test_encodings = tokenizer(test_df["Phrase"].tolist(), padding=True, truncation=True)
val_encodings = tokenizer(validation_df["Phrase"].tolist(), padding=True, truncation=True)

# Convert to PyTorch Dataset
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = MyDataset(train_encodings, train_labels)
test_dataset = MyDataset(test_encodings, test_labels)
val_dataset = MyDataset(val_encodings, val_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2_classification",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IndexError: index out of range in self